In [7]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
import datetime
import pytz

# Function to check if the word "administrator" is in the page content
def contains_administrator(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return "administrator" in response.text.lower()
    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return f"Failed: {e}"

# Function to get all hyperlinks from a webpage
def get_hyperlinks(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return [a['href'] for a in soup.find_all('a', href=True)]
    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return []

# Function to check hyperlinks and nested hyperlinks, then write results to an Excel file
def check_links_and_write_to_excel(start_url):
    # Get the initial list of hyperlinks
    initial_links = get_hyperlinks(start_url)
    initial_links = list(set(initial_links))  # Remove duplicate links
    
    # Initialize Excel workbook
    wb = Workbook()
    ws = wb.active
    ws.append(["Link Address", "Contains 'Administrator'", "Error Message"])

    # Check the response time for each initial link
    for link in initial_links:
        if not link.startswith('http'):
            link = requests.compat.urljoin(start_url, link)
        result = contains_administrator(link)
        if isinstance(result, str) and result.startswith("Failed"):
            ws.append([link, "No", result])  # Record the failure message
        else:
            ws.append([link, "Yes" if result else "No", ""])

        # Get hyperlinks from each page in the initial list and check their response time
        nested_links = get_hyperlinks(link)
        nested_links = list(set(nested_links))  # Remove duplicate links
        for nested_link in nested_links:
            if not nested_link.startswith('http'):
                nested_link = requests.compat.urljoin(link, nested_link)
            nested_result = contains_administrator(nested_link)
            if isinstance(nested_result, str) and nested_result.startswith("Failed"):
                ws.append([nested_link, "No", nested_result])  # Record the failure message
            else:
                ws.append([nested_link, "Yes" if nested_result else "No", ""])

    # Get current date and time
    tz = pytz.timezone('Asia/Jakarta')
    current_datetime = datetime.datetime.now(tz)
    date_string = current_datetime.strftime("%Y%m%d")
    time_string = current_datetime.strftime("%H%M")  # 24-hour format

    # Create file name with date and time
    file_name = f"D:\\Downloads\\link_check_errorfetch_{date_string}_{time_string}.xlsx"
    
    wb.save(file_name)
    print("Process completed successfully :)")

# Example usage
start_url = "https://gis.bnpb.go.id/server/rest/services/"  # Replace this with your starting webpage URL
check_links_and_write_to_excel(start_url)


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Failed to fetch https://gis.bnpb.go.id/server/rest/services/thematic/20240511_022824_75_24d1_3B_AnalyticMS_SR_enhc_tile/ImageServer: 500 Server Error: Internal Server Error for url: https://gis.bnpb.go.id/server/rest/services/thematic/20240511_022824_75_24d1_3B_AnalyticMS_SR_enhc_tile/ImageServer
Process completed successfully :)
